In [2]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 558 ms (started: 2021-08-23 17:05:47 +08:00)


In [3]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #3  Aug 23, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/keras_developer_guides-.git (fetch)
origin	git@github.com:ustchope/keras_developer_guides-.git (push)
[main f760c7b] 更新 #3  Aug 23, 2021
 3 files changed, 2380 insertions(+), 141 deletions(-)
 create mode 100644 The Functional API.ipynb
 create mode 100644 "\351\241\272\345\272\217\346\250\241\345\236\213.ipynb"


To git@github.com:ustchope/keras_developer_guides-.git
   b364734..f760c7b  main -> main


time: 5.96 s (started: 2021-08-23 17:05:49 +08:00)


In [4]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 3.9 s (started: 2021-08-23 17:06:17 +08:00)


# 设置

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

time: 1.16 ms (started: 2021-08-23 17:06:44 +08:00)


# 何时使用顺序模型

Sequential 模型适用于一个普通的层堆栈，其中每一层恰好有一个输入张量和一个输出张量。

示意性地，以下顺序模型：

In [6]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

time: 3.25 s (started: 2021-08-23 17:07:20 +08:00)


相当于这个函数：

In [7]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

time: 22.4 ms (started: 2021-08-23 17:08:01 +08:00)


在以下情况下，顺序模型不合适：
* 您的模型有多个输入或多个输出
* 您的任何层都有多个输入或多个输出
* 你需要做图层共享
* 您需要非线性拓扑（例如残差连接、多分支模型）

# 创建顺序模型

您可以通过将层列表传递给 Sequential 构造函数来创建 Sequential 模型：

In [8]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

time: 8.73 ms (started: 2021-08-23 17:08:58 +08:00)


它的图层可以通过图层属性访问：

In [10]:
model.layers

time: 6.48 ms (started: 2021-08-23 17:09:24 +08:00)


您还可以通过 add() 方法以增量方式创建 Sequential 模型：

In [12]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

time: 9.47 ms (started: 2021-08-23 17:09:54 +08:00)


请注意，还有一个相应的 pop() 方法来删除层：Sequential 模型的行为非常类似于层列表。

In [13]:
model.pop()
print(len(model.layers))  # 2

2
time: 1.13 ms (started: 2021-08-23 17:10:15 +08:00)


另请注意，Sequential 构造函数接受名称参数，就像 Keras 中的任何层或模型一样。 这对于使用语义上有意义的名称注释 TensorBoard 图很有用。

In [14]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

time: 9.58 ms (started: 2021-08-23 17:10:42 +08:00)


# 预先指定输入形状

通常，Keras 中的所有层都需要知道其输入的形状，以便能够创建它们的权重。 因此，当您创建这样的图层时，最初它没有权重：

In [15]:
layer = layers.Dense(3)
layer.weights  # Empty

[]

time: 4.3 ms (started: 2021-08-23 17:11:14 +08:00)


它在第一次对输入调用时创建它的权重，因为权重的形状取决于输入的形状：

In [16]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

[<tf.Variable 'dense_6/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.41448128,  0.3380506 , -0.29851627],
        [-0.23768789, -0.7175736 , -0.246292  ],
        [-0.3601464 , -0.44008276, -0.21690255],
        [-0.64696825, -0.57585347, -0.4932522 ]], dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

time: 14.7 ms (started: 2021-08-23 17:11:36 +08:00)


当然，这也适用于顺序模型。 当您在没有输入形状的情况下实例化 Sequential 模型时，它不是“构建”的：它没有权重（并且调用 model.weights 会导致错误说明这一点）。 当模型第一次看到一些输入数据时创建权重：

In [17]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

Number of weights after calling the model: 6
time: 50.6 ms (started: 2021-08-23 17:12:12 +08:00)


“构建”模型后，您可以调用它的 summary() 方法来显示其内容：

In [19]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (1, 2)                    10        
_________________________________________________________________
dense_8 (Dense)              (1, 3)                    9         
_________________________________________________________________
dense_9 (Dense)              (1, 4)                    16        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________
time: 4.3 ms (started: 2021-08-23 17:12:37 +08:00)


但是，在逐步构建 Sequential 模型以显示迄今为止模型的摘要（包括当前输出形状）时非常有用。 在这种情况下，您应该通过将 Input 对象传递给模型来启动模型，以便它从一开始就知道其输入形状：

In [20]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
time: 31.1 ms (started: 2021-08-23 17:13:04 +08:00)


请注意，`Input`对象不显示为 model.layers 的一部分，因为它不是一个层：

In [22]:
model.layers

time: 2.88 ms (started: 2021-08-23 17:33:33 +08:00)


一个简单的替代方法是将 input_shape 参数传递给第一层：

In [23]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
time: 28.1 ms (started: 2021-08-23 17:33:53 +08:00)


使用这样的预定义输入形状构建的模型始终具有权重（甚至在看到任何数据之前）并且始终具有定义的输出形状。

一般来说，如果您知道序列模型的输入形状是什么，那么推荐的最佳做法是始终提前指定它的输入形状。

# 一个常见的调试工作流程：add() + summary()

在构建新的 Sequential 架构时，使用 add() 递增堆叠层并经常打印模型摘要很有用。 例如，这使您能够监控 Conv2D 和 MaxPooling2D 层的堆栈如何对图像特征图进行下采样：

In [24]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
____________________________

很实用吧？

# 有了模型后该做什么

一旦您的模型架构准备就绪，您将需要：
* 训练您的模型、评估它并运行推理。 请参阅我们的内置循环训练和评估指南
* 将您的模型保存到磁盘并恢复它。 请参阅我们的序列化和保存指南。
* 通过利用多个 GPU 加速模型训练。 请参阅我们的多 GPU 和分布式训练指南。

# 使用序列模型进行特征提取

一旦构建了 Sequential 模型，它的行为就像一个 Functional API 模型。 这意味着每一层都有一个输入和输出属性。 这些属性可以用来做一些巧妙的事情，比如快速创建一个模型，提取 Sequential 模型中所有中间层的输出：

In [29]:
initial_model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

time: 54.1 ms (started: 2021-08-23 17:38:23 +08:00)


这是一个仅从一层提取特征的类似示例：

In [32]:
initial_model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

time: 53.9 ms (started: 2021-08-23 17:40:11 +08:00)


# 使用序列模型进行迁移学习

迁移学习包括冻结模型中的底层并仅训练顶层。 如果您不熟悉它，请务必阅读我们的迁移学习指南。

以下是两个涉及序列模型的常见迁移学习蓝图。

首先，假设您有一个 Sequential 模型，并且您想冻结除最后一层之外的所有层。 在这种情况下，您只需遍历 model.layers 并在除最后一层之外的每一层上设置 layer.trainable = False。 像这样：

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(784)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])

# Presumably you would want to first load pre-trained weights.
model.load_weights(...)

# Freeze all layers except the last one.
for layer in model.layers[:-1]:
  layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
model.compile(...)
model.fit(...)

另一个常见的蓝图是使用 Sequential 模型来堆叠一个预先训练好的模型和一些新初始化的分类层。 像这样：

In [ ]:
# Load a convolutional base with pre-trained weights
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# Freeze the base model
base_model.trainable = False

# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])

# Compile & train
model.compile(...)
model.fit(...)

如果您进行迁移学习，您可能会发现自己经常使用这两种模式。

这就是您需要了解的有关序列模型的全部内容！

要了解有关在 Keras 中构建模型的更多信息，请参阅：
* 函数式 API 指南
* 通过子类创建新层和模型的指南